# Generate a Grid of Matrices (labelled by $w_0,R_0,Z$) that can be used to get the CR ring for Arbitrary LP and CP 
---

**Contact: oiangu9@gmail.com**

In this notebook we explain how to generate a dataset of matrices that yield simulated non-noisy/theoretical conical refraction (CR) rings. 

**Concept**

As explained in A.2 and A.3 of the bachelor thesis, the standard set-up to observe CR (where the input light beam is a cylindrically symmetric gaussian beam and a convergent lens is placed before the crystal), has explicit mathematical formulas that allow us to simulate the phenomenon as a function of some laboratory parameters (or their idelaizaitons). 

The parameters that allow to sweep all the possible rings are (all in pixel units):

- $Z$: The theoretical distance from the camera plane to the focal plane of the CR ring.
- $w_0$: The theoretical waist of the input beam (the diameter of the beam at the focal plane in the abscense of crystal).
- $R_0$: The geometric radius of the ring in the focal plane.

Then, as explained in A.3, using equations (37)-(42) from A.2, we can compute the intensity profile for any input polarization of light.

The key point is that computing the matrices that depend on $Z,w_0,R_0$ is very computationally costly because some Bessel function computations and big numerical integrals need to be taken. Fortunately, it so happens that we only need to make explicit the polarization of the input light after this costly computations, and using the same matrix we can obtain the CR ring image for any polarization very cheaply. Therefore, the idea is to pre-compute the matrices for different $Z,w_0,R_0$ settings, store them in a hard drive and then, whenever we need them, instead of computing them, retrieve them from the storage and apply the last steps to get the image for a particular polarization.

This then allows to generate cheap CR images in real time, instead of needing to wait until the integrals are computed for each $Z,w_0,R_0$ we want to check (very useful to do a real time simulation fitting etc.). With the drawback that only those combinations of $Z,w_0,R_0$ available in the precomputed dataset will be accessible.

Hence, the idea is to generate a dataset of all the $Z,w_0,R_0$ we can. Certainly we cannot pre-compute them for the whole continuum of values. Instead, we will generate them for a lattice of interesting $Z,w_0,R_0$. 

Hereafter, we will only be interested in circularly and linearly polarized light.

**Modus Operandi**

 - You will need to specify a box for the space of $Z,w_0,R_0$ to be considered. This box will be discretized into a lattice of $N_Z\times N_{w_0}\times N_{R_0}$ points (also specified by you), ideally as high as feasible. <p> 


- Then the idea is to generate a file of format `.h5` that will store the precomputation matrices (as numpy matrices), indexed by some label ID, and a `.json` file (hence, a Python dictionary), that will tell which is the `Z,w_0,R_0` combination of each label ID in the `.h5`. <p> 


- The calculations are done using the GPU (if available $-$make sure it is) and the library `jax`, which employs exactly the same syntax as numpy but uses the GPU for calculations. <p> 


 We will do the calculations in parallel (if wished, otherwise just set the number of workers to 1). The parallel executions can run in the same or different computers. The way the notebook works is the following one:
 
 1. Fix all the parameters for the lattice in Section 1.<p> 
 
 2. Choose the number of parts in which you want to split the task (so-called, the number of workers). Each worker (in the same or different computers) will compute a portion of the lattice, independently of whether the other parts have been computed or not. Note that if you put too many workers using the same GPU, it might run out of memory (the python script will let you know if such is the case).<p> 
 
 2. Make a copy of this notebook for each worker.<p> 
 
 3. For each worker, change the variable telling the script which worker it is in Section 2, and run the generation of matrices of Section 3.<p> 
 
 4. When every worker is done, move all the generated `.h5` and `.json` pairs to the same directory (using a pen drive if needed to move files from another computer). Now using a single worker notebook, run Section 4 to merge all the dataset into a full dataset.

It is simpler than it seems at first read, really!
     
**What exactly these datasets contain and How to use them**

Go to Section 5.
     
**What if I wanted a dataset of actual `.png` images and not the matrices?**

Go to Section 6.
   

## Section 1: Common Parameters for all Workers
Set these parameters before creating the different workers.

In [2]:
# Give a name to the dataset #############################################################
experiment_name = "test"

# Choose the directory where the dataset will be outputed
# (if non-existent, it will be created by the script)
output_directory = f"./OUTPUT/"

# The simulated images will be square (length=width): #####################################
# Choose the size of the images (and the output matrices)
image_side = 521   # for tests use 81
# Warning: it should be ODD (2X+1), and ideally should match 
# the short side of the camera images with which they will be compared


# Choose the bound for the box in the R0,w0,Z space #######################################
# (recall, they are in pixel units)
min_R0 = 100 # for tests use 10
max_R0 = 180 # for tests use 30

min_w0 = 8  # for tests 1
max_w0 = 40 # for tests 8

min_Z = 0 
max_Z = 0.1
# choose the bounds for Z in the positive range, because the 
# ring at Z=a and Z=-a is the same!


# Choose the number of points per axis in the lattice ########################################
N_R0 = 100   # number of points along the R0 axis  
N_w0 = 100    # number of points along the w0 axis
N_Z = 4      # number of points in the Z axis
# for tests use e.g. N_R0=5 N_w0=5 N_z=2
# Note: the time to be waited and the weight of the resulting files
# will increase exponentially as you increase any of the Ns

# Choose how often you want the progess bar to be updated ###################################
# (if too small, this will slow down the generation)
output_info_every = 25 #many lattice points

# Choose how often the generated matrices and json should be dumped to the external file ######
# (if too small, this will slow down the generation)
dump_every = 25 #many lattice points
# The trick is that if we keep dumping to the external file, halting manually the calculations
# by clicking the square button will not erase the made progress. Even if we shut down the computer,
# just click run again to continue the generation of the dataset


# The following need not be changed a priori ###############################################
# Maximum frequency magnitude for the integration in the Fourier space
max_k = 50
# Number of points to consider for the Fourier space integral 
num_k = 1200
# Randomization seed (unless this changes, the same random 
# decisions will be taken at each run of the script)
randomization_seed = 666

## Section 2: Tell this script which is their worker number and their worker ID

In [3]:
# Which is the total number of workers that will compute this?
TOTAL_WORKERS = 3
# Who am I (which is my worker ID)?
WORKER_ID = 0
# Warning: begin from 0 till TOTAL_WORKERS-1

################################################################
def beg_index(N,j,K):
    '''N: number of tasks (R0_w0_Z triplets to compute);
       j: a worker ID;    K: total number of workers'''
    return (N//K)*j + j*((N%K-j)>0) + (N%K)*(j>=N%K)
def end_index(N,j,K):
    return (N//K)*(j+1) + (j+1)*((N%K-j-1)>0)+ (N%K)*(j+1>=N%K)

print(f"I am worker number {WORKER_ID} and out of the {N_R0*N_w0*N_Z} matrices to be computed,")
print(f"I am going to compute from the {beg_index(N_R0*N_w0*N_Z,WORKER_ID,TOTAL_WORKERS)}-th one to the {end_index(N_R0*N_w0*N_Z,WORKER_ID,TOTAL_WORKERS)}-th one.")

I am worker number 0 and out of the 200 matrices to be computed,
I am going to compute from the 0-th one to the 67-th one.


## Section 3: Run the Calculations of this Worker

In [ ]:
# Import packages ######################################
import os
# This .py file should be in the same directory!
from CLASS_GPU_Simulator import *
import numpy as np
import json
from IPython import display
from time import time
import h5py

# General preambles ######################################
os.makedirs(output_directory, exist_ok=True)
np.random.seed(randomization_seed)

# Generate lattice axes
R0_s= np.linspace(min_R0, max_R0, N_R0)#np.linspace(110, 180, N_R0) #np.linspace(70,180,40) # in pxels 153
w0_s= np.linspace(min_w0, max_w0, N_w0)#np.linspace( 8, 35, N_w0) #np.linspace(8,50,40) 11
Z_s=np.linspace(min_Z, max_Z, N_Z)

# This will tell the worker how to compute only those matrices in a fixed range:
# we distribute/chop one of the axes of the lattice among the workers 
# (If you do it with all axes you do not get the whole lattice covered!)
R0_s = R0_s[beg_index(N_R0, WORKER_ID, TOTAL_WORKERS):end_index(N_R0, WORKER_ID, TOTAL_WORKERS)]

i=1 # Counter
total=Z_s.shape[0]*R0_s.shape[0]*w0_s.shape[0] # total number of elements to compute
elapsed=0  # time elapsed since computations started
beg=time() # initial time



# Initialize index bookkeeper ###########################
try: # If the computation was halted by the user, then instead of starting from zero, continue wherever we were before
    index_bookkeeper = json.load(open(f"{output_directory}/BOOKKEEPER_PART_{WORKER_ID}_{experiment_name}.json"))
except: # Else, start from scratch
    index_bookkeeper = {'R0s':[], 'w0s':[], 'Zs':[], 'IDs':[]}

# Initialize Simulator object ############################
simulator =RingSimulator_GPU( n=1.5, a0=1.0, max_k=max_k, num_k=num_k, nx=image_side, 
                                      sim_chunk_x=image_side, sim_chunk_y=image_side)

# Initialize the hdf5 dataset storing file ###############
# If it already existed (because the user halted the generation),
# we will append the next images, else create a new file
h5f = h5py.File(f"{output_directory}/DATASET_PART_{WORKER_ID}_{experiment_name}.h5", 'a') 


try: # Store in the h5f the angle at which each pixel is located relative to the central pixel
    h5f.create_dataset('phis', data=simulator.phis[:,:,0], compression="lzf", shuffle=True) #, compression_opts=9)
except: 
    h5f['phis'][:] = simulator.phis[:,:,0]

# Run the generation loop #####################
for Z in Z_s:
    for R0 in R0_s:
        for w0 in w0_s:
            ID = f"R0_{str(R0)}_w0_{str(w0)}_Z_{str(Z)}" # this will be the ID for the matrices in the json
            if ID not in index_bookkeeper['IDs']: # then it has not been simulated yet
                # simulate matrix with this R0,w0,Z
                D_matrix = simulator.compute_pieces_for_I_LP_and_CP(R0_pixels=R0, Z=Z, w0_pixels=w0)
                
                if D_matrix is None: # there has been some error. Stop the execution
                    raise ValueError
                    
                try: # Store the generated matrix in the hdf5 file
                    h5f.create_dataset(ID, data=D_matrix, compression="lzf", shuffle=True) #, compression_opts=9)
                except: # in case the index_bookkeeper did not record it, but it was already in h5f
                    print(f"{ID} was already in h5f but not in index_bookkeeper")
                    h5f[ID][:] = D_matrix

                # Append the index and features of the generated matrix to the bookkeeping dicitonary
                index_bookkeeper['IDs'].append(ID)
                index_bookkeeper['R0s'].append(str(R0))
                index_bookkeeper['Zs'].append(str(Z))
                index_bookkeeper['w0s'].append(str(w0))
                
                # Print state to user
                if i%output_info_every==0:
                    display.clear_output(wait=True)
                    elapsed=time()-beg
                    print(f"["+'#'*(int(100*i/total))+' '*(100-int(100*i/total))+f"] {100*i/total:3.4}% \n\nSimulated: {i}/{total}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")
                
                # Store the progress in the external files (in order to be able to quit and resume)
                if i%dump_every==0:
                    h5f.flush() # technically the matrices are saved already, but need to flush just in case
                    json.dump(index_bookkeeper, open( f"{output_directory}/BOOKKEEPER_PART_{WORKER_ID}_{experiment_name}.json", "w"))
            i+=1
            
# Print Final log details
display.clear_output(wait=True)
elapsed=time()-beg
print(f"["+'#'*(int(100*i/total))+' '*(100-int(100*i/total))+f"] {100*i/total:3.4}% \n\nSimulated: {i}/{total}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")               

# Finall flush of data to files
h5f.flush()
json.dump(index_bookkeeper, open( f"{output_directory}/BOOKKEEPER_PART_{WORKER_ID}_{experiment_name}.json", "w"))

# Close hdf5 properly
h5f.close()
print(f"\n\nWORKER {WORKER_ID} FINISHED!!!")

**Important**: While the previous cell is running, you CAN interrupt the calculations (e.g., by clicking the square button above). The matrices calculated until the last update of the progress bar are safely stored if you run the next cell to close the dataset. Then, even if you shutdown the computer, the next time you run the code, the calculations will continue where they were left.

In [ ]:
# Will give error if you waited the whole computation of the previous cell
h5f.flush()
h5f.close()

## Section 4: Join the pieces calculated by the different workers!
**Only ONE of the workers must run the following!**

Run Section 1 and 2 and then the next cell.

In [ ]:
import os
import numpy as np
import json
import h5py

# Create the big h5f and bookkeeper that will contain all of the parts
h5f = h5py.File(f"{output_directory}/DATASET_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.h5", 'w') # append if exists, create if not
total_bookkeeper = {'R0s':[], 'w0s':[], 'Zs':[], 'IDs':[]}

for j in range(TOTAL_WORKERS):
    # open worker j's part of the dataset
    h5f_worker = h5py.File(f"{output_directory}/DATASET_PART_{j}_{experiment_name}.h5", 'r')
    worker_bookkeeper = json.load(open(f"{output_directory}/BOOKKEEPER_PART_{j}_{experiment_name}.json"))
    if j==0:
        h5f.create_dataset('phis', data=h5f_worker['phis'][:], compression="lzf", shuffle=True)
    for ID in worker_bookkeeper['IDs']:
        h5f.create_dataset(ID, data=h5f_worker[ID][:], compression="lzf", shuffle=True)
    # dump all the results to the total dataset holders
    total_bookkeeper['R0s'] = total_bookkeeper['R0s'] + worker_bookkeeper['R0s']
    total_bookkeeper['w0s'] = total_bookkeeper['w0s'] + worker_bookkeeper['w0s']
    total_bookkeeper['Zs'] = total_bookkeeper['Zs'] + worker_bookkeeper['Zs']
    total_bookkeeper['IDs'] = total_bookkeeper['IDs'] + worker_bookkeeper['IDs']
    h5f_worker.close()
    print(f"Worker {j}'s job transferred to total dataset files!")
json.dump(total_bookkeeper, open( f"{output_directory}/BOOKKEEPER_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.json", "w"))
h5f.close()
print(f"Total dataset generated! It has two files: \n  >{output_directory}/DATASET_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.h5\n  >{output_directory}/BOOKKEEPER_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.json")

Run the following cell if you want to erase all the partial calculation files (not needed anymore, so you can safely erase them!)

In [6]:
for j in range(TOTAL_WORKERS):
    os.remove(f"{output_directory}/DATASET_PART_{j}_{experiment_name}.h5")
    os.remove(f"{output_directory}/BOOKKEEPER_PART_{j}_{experiment_name}.json")

## Section 5: How to use the dataset and what it contains exactly

You have generated two files:

- (a) `BOOKKEEPER_....json` (which is quite light)

    You can open it in python using
    
    `import json`
    
    `index_bookkeeper = json.load(open("BOOKKEEPER_....json"))`
    
    It is just a python dictionary that has four lists inside:
    
    `'R0s':[], 'w0s':[], 'Zs':[], 'IDs':[]`<p>
    
    
    - The first three give the $R_0,w_0,Z$ parameters of each matrix in the (b) file. The values of these lists are saved as strings instead of numbers, so if you want to use them as numbers do not forget to put `float()` around.
    
    - The last list (IDs) gives the exact label used to index the matrices in the (b) file (corresponding to the $R0,w_0,Z$ of the same "row" $-$if you imagine the dictionary as a table).<p>
    
    

- (b) `DATASET_....h5` (which is quite heavy)
        
    You can open it using:
        
        
    `h5_dataset = h5py.File("DATASET_....h5", 'r')`
        
        
    Let's say you are interested in the matrix with index `ID` (e.g., `index_bookkeeper['IDs'][k]` for the `k`-th matrix). 
        You can extract it to a numpy array as:
        
        
    `my_matrix = h5_dataset[ID][:]`
        
    In addition, the `.h5` file also contains a precomputed matrix of size `[image_size, image_size]` giving the polar angle (in radians) of each pixel in an image of size `[image_size, image_size]`, relative to the central pixel (recall `image_size` was required to be odd!). This matrix can be retrieved as:
        
        
    `my_matrix = h5_dataset['phis'][:]`
        
    When you are done using the dataset close it using
        
    `h5_dataset.close()`

**What do these matrices contain?**
        
For the fixed target image resolution `[image_side, image_side]` that you chose and for all $R_0,w_0,Z$ in the given lattice, each matrix contains the result for a particular combiantion of $R_0,w_0,Z$ of 

    np.stack((
        B0.real**2+B0.imag**2 + B1.real**2+B1.imag**2, 2*(B1.conjugate()*B0).real
    ))

where `B0`, `B1` are the terms with the same names of equation (40) and (42) in the bachelor's thesis. Here, `B0` and `B1` are 2D matrices of size `[image_side, image_side]`. 

Then, if we denote by `D` the result of the last computation (not to confuse with the D of the thesis), and if we denote by `phi_grid` a 2D matrix with shape `[image_side, image_side]` that tells the polar angle of each pixel relative to the central pixel (e.g., `h5_dataset['phis'][:]`), one can then obtain the linear polarization CR ring image for an angle `phiCR` in radians by computing, following equation (42):

     D[0]+D[1]*np.cos(phiCRs-phis) #[image_side, image_side]
    
On the other hand, `D[0]` already gives the circular polarization's image.
        
Note that it might be interesting (depending of what you are going to do), to normalize the resulting images, so that the maximum intensity pixel takes the maximum value (i.e., divide the image matrix element-wise by the maximum intensity value).
    
That is, what **the "D-matrices" dataset contains is exactly the precursor to the last (silly) calculation to obtain a simulated image** with linear polarization, and the full calculation for circular polarization.


Here some example usage, retrieval of images etc. (run Section 1 and 2 and then the next cell):

In [ ]:
import json
import h5py
import matplotlib.pyplot as plt
import numpy as np

# write here the path to the datasets
path_to_h5 =      # f"{output_directory}/DATASET_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.h5"
path_to_json =      # f"{output_directory}/BOOKKEEPER_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.json"

# open the two files
h5_dataset = h5py.File(path_to_h5, 'r')
index_bookkeeper = json.load(open(path_to_json))

# retrieve the matrix with the polar angles at each pixel
phis = h5_dataset['phis'][:]

# to retrieve the first element of the dataset
# first take its ID label
ID_first_element = index_bookkeeper['IDs'][20]

# now access it
example_D_matrix = h5_dataset[ID_first_element][:]

# generate the circular polarization image
CP_image = example_D_matrix[0] #[image_side, image_side]


# fix a linear polarization angle
example_LP_angle = 3.1415/4
LP_image = example_D_matrix[0]+example_D_matrix[1]*np.cos(example_LP_angle-phis) #[image_side, image_side]

# plot resulting images
print(f"Circular Polarization\nR0={index_bookkeeper['R0s'][0]} pix; w0={index_bookkeeper['w0s'][0]} pix; Z={index_bookkeeper['Zs'][0]} pix")
plt.imshow(CP_image)
plt.show()

print(f"Linear Polarization at phi={example_LP_angle}rad\nR0={index_bookkeeper['R0s'][0]} pix; w0={index_bookkeeper['w0s'][0]} pix; Z={index_bookkeeper['Zs'][0]} pix")
plt.imshow(LP_image)
plt.show()

# close the dataset when finished!
h5_dataset.close() 

In [ ]:
# A little loop to show how the same D matrix allows the generation of any linear polarization
for LP_angle in np.linspace(0,2*3.1415, 10):
    # generate the image for this angle using the same D matrix
    LP_image = example_D_matrix[0]+example_D_matrix[1]*np.cos(LP_angle-phis) #[image_side, image_side]
    print(f"Linear Polarization at phi={LP_angle:.5} rad\nR0={index_bookkeeper['R0s'][0]} pix; w0={index_bookkeeper['w0s'][0]} pix; Z={index_bookkeeper['Zs'][0]} pix")
    plt.imshow(LP_image)
    plt.show()

In [ ]:
# What to do if I wanted to save the resulting image as a .png for some other stuff?
import cv2

image_with_8bits_per_pixel = ((2**8-1)*LP_image/np.max(LP_image)).astype(np.uint8) 
cv2.imwrite("example_image_8bits.png", image_with_8bits_per_pixel)

image_with_16bits_per_pixel = ((2**16-1)*LP_image/np.max(LP_image)).astype(np.uint16) 
cv2.imwrite("example_image_16bits.png", image_with_16bits_per_pixel)
# you can check that the file of 16 bits occupies more memory than the 8 bit one

# What to do if I want to read such an image back to python?
image_with_8bits_per_pixel = cv2.imread("example_image_8bits.png", flags=cv2.IMREAD_ANYDEPTH + cv2.IMREAD_GRAYSCALE)
image_with_16bits_per_pixel = cv2.imread("example_image_16bits.png", flags=cv2.IMREAD_ANYDEPTH + cv2.IMREAD_GRAYSCALE)

print(image_with_16bits_per_pixel.dtype, image_with_16bits_per_pixel.shape)
print(image_with_8bits_per_pixel.dtype, image_with_8bits_per_pixel.shape)

# Take them to floats if you want to do operations with them!
image_8_in_floats = image_with_8bits_per_pixel.astype(np.float32) # or float64 is double precision required!
image_16_in_floats = image_with_16bits_per_pixel.astype(np.float32)

print(image_with_8bits_per_pixel[:3,:3])
print(image_8_in_floats[:3,:3])

In [50]:
# Remove the test images we just generated!
import os
os.remove("example_image_8bits.png")
os.remove("example_image_16bits.png")

## Section 6: How to make a `.png` dataset out of this?

Maybe you prefer to have a folder full of `.png` of simulated CR ring images. I warn you that:

1. You will need to discretize also the linear polarization, so you will loose the possibility to access the full continuum, and since the angle of polarization is the parameter we are interested on, this might not be a good idea unless you make the angle grid dense enough.<p>

2. This will occupy way more space than the previous dataset, because you will need to consider now all the previous dataset for each point in the angle grid (so multiply the weight of the `h5`by the number of angle points you want to consider, to estimate the storage space needed for that).
    
If you are okay with that, just fill the parameters in the next cell and run the following one.

In [1]:
# Select the OUTPUT path. The images will be generated to a sub-folder (if non-existent path, it will be created)
output_directory = "./OUTPUT/"
experiment_name = "Test"

# Select if you want to generate images of circular and/or linear polarization ###############
circular_polarization = True
linear_polarization = True

# Choose the grid in linear polarization angles you are looking for ##########################
N_phiCR = 10

# write here the path to the datasets ########################################################
path_to_h5 =    # f"{output_directory}/DATASET_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.h5"
path_to_json =   # f"{output_directory}/BOOKKEEPER_R0_{N_R0}_w0_{N_w0}_Z_{N_Z}_{experiment_name}.json"

# print progess bar every x many generated images
log_every = 5

Run following code:

In [16]:
import numpy as np
import json
import h5py
import cv2
import os
from IPython import display

# open the two files
h5_dataset = h5py.File(path_to_h5, 'r')
index_bookkeeper = json.load(open(path_to_json))

# generate grid of angles
phiCRs = np.linspace( 0, 2*np.pi, N_phiCR )

# retrieve the matrix with the polar angles at each pixel
phis = h5_dataset['phis'][:]

total = len(index_bookkeeper['IDs'])
# make output directory if non-existent
output_path = output_directory + f"/IMAGE_DATASET_{experiment_name}_N_R0_{len(set(index_bookkeeper['R0s']))}_N_w0_{len(set(index_bookkeeper['w0s']))}_N_Z_{len(set(index_bookkeeper['Zs']))}_N_phiCR_{N_phiCR}"
os.makedirs(output_path, exist_ok=True)

for i,ID in enumerate(index_bookkeeper['IDs']):
    # retrieve D matrix
    D_mat = h5_dataset[ID][:]
    if linear_polarization:
        for LP_angle in phiCRs:
            LP_image = D_mat[0]+D_mat[1]*np.cos(LP_angle-phis) #[image_side, image_side]
            cv2.imwrite(f"{output_path}/{ID}_LP_angle_{str(LP_angle)}.png", ((2**8-1)*LP_image/np.max(LP_image)).astype(np.uint8))
    if circular_polarization:
        cv2.imwrite(f"{output_path}/{ID}_CP.png", ((2**8-1)*D_mat[0]/np.max(D_mat[0])).astype(np.uint8))
    if i%log_every==0:
        display.clear_output(wait=True)
        print(f"["+'#'*(int(100*(i+1)/total))+' '*(100-int(100*i/total))+f"] {100*(i+1)/total:3.4}% \n\Generated Images: {i+1}/{total}\n")               

display.clear_output(wait=True)
print(f"["+'#'*(int(100*(i+1)/total))+' '*(100-int(100*i/total))+f"] {100*(i+1)/total:3.4}% \n\Generated Images: {i+1}/{total}\n")               
h5_dataset.close()

[#################################################################################################### ] 100.0% 
\Generated Images: 200/200

